In [1]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# Initialising the CNN
classifier = Sequential()

In [3]:
# Step 1 - Convolution
# 32 - number of output filters
# Rectified Linear Unit as activation function
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))



In [4]:
# Step 2 - Pooling
#pool size - (2,2)
classifier.add(MaxPooling2D(pool_size = (2, 2)))



In [5]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))



In [6]:
# Step 3 - Flattening
classifier.add(Flatten())

In [7]:
# Step 4 - Full connection
# units - Dimensionality of the output space
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [8]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = {"output_a":'accuracy'})



In [9]:
from keras.preprocessing.image import ImageDataGenerator
import scipy.ndimage

#shear_range: Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees)
#zoom_range: Float or [lower, upper]. Range for random zoom.

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/Users/saarthak/Desktop/cat-and-dog/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/Users/saarthak/Desktop/cat-and-dog/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set, steps_per_epoch = 4000,epochs = 20,
                         validation_data = test_set,validation_steps = 2000)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Epoch 1/2
4000/4000 [==============================] - 2585s 646ms/step - loss: 0.1579 - val_loss: 0.1429
Epoch 2/2
4000/4000 [==============================] - 2620s 655ms/step - loss: 0.0758 - val_loss: 0.1459


In [11]:
# Part 3 - Making new predictions
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/Users/saarthak/Desktop/cat-and-dog/single_prediction/cat_or_dog_12.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
print(result)
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

[[ 0.]]
cat


In [31]:
classifier.save_weights('weights.h5')